In [1]:
from app.utils.t5 import *
torch.cuda.empty_cache()
pl.seed_everything(42)

Global seed set to 42


42

In [11]:
torch.cuda.get_device_name(0)

'A100-SXM4-40GB'

In [3]:
!pip freeze > requirements.txt

In [12]:
input_data_name = "preprocessed.csv"                            
data_inpit_dir = "./Data/Preprocessed/"                              
output_dir = "./Data/Model/"
source_column = "outline"    
target_column =  "original_text"   
model_name_str = "t5-base_grammar"

no_workers = 1

imput_data_path = data_inpit_dir + input_data_name
new_model_name = "d-t5-{}".format(model_name_str)

In [13]:
data = pd.read_csv(imput_data_path)

data['target_text'] = data[target_column]
data['source_text'] = "grammar: " + data[source_column]
#data['source_text'] = data[source_column]

data['target_len'] = data["target_text"].str.split().str.len()
data['source_len'] = data["source_text"].str.split().str.len()

descriptives = data[['target_len','source_len']].describe()
descriptives

,target_len,source_len
count,281.000000,281.000000
mean,176.921708,156.252669
std,52.903898,53.038701
min,53.000000,42.000000
25%,140.000000,117.000000
50%,169.000000,151.000000
75%,211.000000,187.000000
max,332.000000,320.000000


In [14]:
train = data[data.type == "train"][["target_text", "source_text"]]
dev = data[data.type == "dev"][["target_text", "source_text"]]

In [15]:
model = SimpleT5()

In [7]:
model.from_pretrained(model_type="t5",model_name = "t5-base") # large  "google/mt5-base" byt5-base vennify/t5-base-grammar-correction

In [8]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [9]:
model.train(train_df = train, 
            eval_df = dev, 
            source_max_token_len = 320,   
            target_max_token_len = 332,  
            batch_size = 2, max_epochs = 10, # 20  9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 1,
            new_model_name = new_model_name) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-cd2a3fdb-d296-06a1-ff6a-5b6d54df0acc]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 737 M 
-----------------------------------------------------
737 M     Trainable params
0         Non-trainable params
737 M     Total params
2,950.672 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Global seed set to 42
/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:322: UserWarning: The number of training samples (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.016 >= min_delta = 0.0. New best score: 1.540


Average training loss for epoch 1 equal to 0.9054


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 1 records. Best score: 1.540. Signaling Trainer to stop.


Average training loss for epoch 2 equal to 0.5058
